# Laden der Brustkrebsdaten in die SQLite-Datenbank

In [74]:
#Laden Libraries
import sqlite3
import pandas as pd

In [75]:
# Verbindung zur Datenbank
conn = sqlite3.connect('BreastCancerDB.db')
cursor = conn.cursor()
print("Datenbank wurde erfolgreich geöffnet");

Datenbank wurde erfolgreich geöffnet


In [76]:
# Löschen Tabellen wenn sie schon exestieren
cursor.execute("DROP TABLE IF EXISTS patients")
cursor.execute("DROP TABLE IF EXISTS conditions")
cursor.execute("DROP TABLE IF EXISTS medications")
cursor.execute("DROP TABLE IF EXISTS observations")
cursor.execute("DROP TABLE IF EXISTS procedures")
cursor.execute("DROP TABLE IF EXISTS patient_careplans")
cursor.execute("DROP TABLE IF EXISTS patients_new")

In [77]:
# Laden Daten
patients = pd.read_csv('BreastCancerData/patients.csv', sep=",")
conditions = pd.read_csv('BreastCancerData/conditions.csv', sep=",")
medications = pd.read_csv('BreastCancerData/medications.csv', sep=",")
observations = pd.read_csv('BreastCancerData/observations.csv', sep=",")
procedures = pd.read_csv('BreastCancerData/procedures.csv', sep=",")

In [78]:
# Tabellen erstellen
cursor.execute('''
		CREATE TABLE IF NOT EXISTS patients (
			Id nvarchar(36) primary key,
            BIRTHDATE Date,
            DEATHDATE Date,
            SSN nvarchar(50),
            DRIVERS nvarchar(50),
            PASSPORT nvarchar(10),
            PREFIX nvarchar(4),
            FIRST nvarchar(50),
            LAST nvarchar(50),
            SUFFIX nvarchar(10),
            MAIDEN nvarchar(50),
            MARITAL nvarchar(1),
            RACE nvarchar(10),
            ETHNICITY nvarchar(50),
            GENDER nvarchar(1),
            BIRTHPLACE nvarchar(50),
            ADDRESS nvarchar(50),
            CITY nvarchar(50),
            STATE nvarchar(50),
            COUNTY nvarchar(50),
            ZIP nvarchar(4),
            LAT nvarchar(10),
            LON nvarchar(10),
            HEALTHCARE_EXPENSES float,
            HEALTHCARE_COVERAGE float
          
			)
             ''')

cursor.execute('''
		CREATE TABLE IF NOT EXISTS conditions (
            START Date,
            STOP Date,
            PATIENT nvarchar(36),
            ENCOUNTER nvarchar(50),
            DESCRIPTION nvarchar(256),
            CODE int,
            foreign key(PATIENT) references patients(Id) 
			)
               ''')

cursor.execute('''
		CREATE TABLE IF NOT EXISTS medications(
            START nvarchar(20),
            STOP nvarchar(20),
            PATIENT nvarchar(36),
            PAYER nvarchar(50),
            ENCOUNTER nvarchar(50),
            CODE int,
            DESCRIPTION nvarchar(256),
            BASE_COST float,
            PAYER_COVERAGE float,
            DISPENSES int,
            TOTALCOST float,
            REASONCODE float,
            REASONDESCRIPTION nvarchar(50),
            foreign key(PATIENT) references patients(Id) 
			)
               ''')

cursor.execute('''
		CREATE TABLE IF NOT EXISTS observations (
            DATE nvarchar(20),
            PATIENT nvarchar(36),
            ENCOUNTER nvarchar(50),
            CODE int,
            DESCRIPTION nvarchar(256),
            VALUE float,
            UNITS nvarchar(10),
            TYPE nvarchar(10),
            foreign key(PATIENT) references patients(Id) 
			)
               ''')

cursor.execute('''
		CREATE TABLE IF NOT EXISTS procedures(
            DATE nvarchar(20),
            PATIENT nvarchar(36),
            ENCOUNTER nvarchar(50),
            CODE int,
            DESCRIPTION nvarchar(50),
            BASE_COST float,
            REASONCODE float,
            REASONDESCRIPTION nvarchar(256),
            foreign key(PATIENT) references patients(Id) 
			)
               ''')

In [79]:
# Einfügen der Daten in die erstellten Tabellen
patients.to_sql('patients', conn, if_exists='append', index=False)
conditions.to_sql('conditions', conn, if_exists='append', index=False)
medications.to_sql('medications', conn, if_exists='append', index=False)
observations.to_sql('observations', conn, if_exists='append', index=False)
procedures.to_sql('procedures', conn, if_exists='append', index=False)

# Sternschema erstellen

Wir benötigen eine Faktentabelle für unser Sternschema. Die Tabellen patients, medications, observations und procedures werden als Dimensionstabelle für das Sternschema genommen. Die neue Tabelle erhalten den Namen mit einem Präfix _new und sie werden mit den Daten aus der alten Tabellen befüllt.

In [80]:
patients.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1019 entries, 0 to 1018
Data columns (total 25 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Id                   1019 non-null   object 
 1   BIRTHDATE            1019 non-null   object 
 2   DEATHDATE            19 non-null     object 
 3   SSN                  1019 non-null   object 
 4   DRIVERS              854 non-null    object 
 5   PASSPORT             803 non-null    object 
 6   PREFIX               831 non-null    object 
 7   FIRST                1019 non-null   object 
 8   LAST                 1019 non-null   object 
 9   SUFFIX               15 non-null     object 
 10  MAIDEN               280 non-null    object 
 11  MARITAL              689 non-null    object 
 12  RACE                 1019 non-null   object 
 13  ETHNICITY            1019 non-null   object 
 14  GENDER               1019 non-null   object 
 15  BIRTHPLACE           1019 non-null   o

In [81]:
cursor.execute('''
		CREATE TABLE IF NOT EXISTS patients_new(
			Id nvarchar(36) primary key,
            BIRTHDATE_YEAR Date,
            DEATHDATE Date,
            SSN nvarchar(50),
            SUFFIX nvarchar(10),
            MAIDEN nvarchar(50),
            MARITAL nvarchar(1),
            RACE nvarchar(10),
            ETHNICITY nvarchar(50),
            GENDER nvarchar(1),
            BIRTHPLACE nvarchar(50),
            CITY nvarchar(50),
            STATE nvarchar(50),
            COUNTY nvarchar(50),
            ZIP nvarchar(4),
            LAT nvarchar(10),
            LON nvarchar(10),
            HEALTHCARE_EXPENSES float,
            HEALTHCARE_COVERAGE float
          
			)
             ''')

cursor.execute('''
		CREATE TABLE IF NOT EXISTS conditions_new (
            CONDITIONS_ID INTEGER PRIMARY KEY AUTOINCREMENT,
            START Date,
            STOP Date,
            PATIENT nvarchar(36),
            ENCOUNTER nvarchar(50),
            DESCRIPTION nvarchar(256),
            CODE int,
            foreign key(PATIENT) references patients_new(Id) 
			)
               ''')             

cursor.execute('''
		CREATE TABLE IF NOT EXISTS observations_new (
            OBSERVATIONS_ID INTEGER PRIMARY KEY AUTOINCREMENT,
            DATE nvarchar(20),
            PATIENT nvarchar(36),
            ENCOUNTER nvarchar(50),
            CODE int,
            DESCRIPTION nvarchar(256),
            VALUE float,
            UNITS nvarchar(10),
            foreign key(PATIENT) references patients_new(Id) 
			)
               ''')  

cursor.execute('''
		CREATE TABLE IF NOT EXISTS procedures_new(
            PROCEDURES_ID INTEGER PRIMARY KEY AUTOINCREMENT,
            DATE nvarchar(20),
            PATIENT nvarchar(36),
            ENCOUNTER nvarchar(50),
            CODE int,
            DESCRIPTION nvarchar(50),
            BASE_COST float,
            REASONCODE float,
            REASONDESCRIPTION nvarchar(256),
            foreign key(PATIENT) references patients_new(Id) 
			)
               ''') 

cursor.execute('''
		CREATE TABLE IF NOT EXISTS medications_new (
            MEDICATIONS_ID INTEGER PRIMARY KEY AUTOINCREMENT,
            START nvarchar(20),
            STOP nvarchar(20),
            PATIENT nvarchar(36),
            PAYER nvarchar(50),
            ENCOUNTER nvarchar(50),
            CODE int,
            DESCRIPTION nvarchar(256),
            BASE_COST float,
            PAYER_COVERAGE float,
            DISPENSES int,
            TOTALCOST float,
            REASONCODE float,
            REASONDESCRIPTION nvarchar(50),
            foreign key(PATIENT) references patients_new(Id) 
			)
               ''')                        

# Faktentabelle 
cursor.execute('''
		CREATE TABLE IF NOT EXISTS patient_careplans (
            CAREPLANT_ID INTEGER PRIMARY KEY AUTOINCREMENT,
			PATIENT_ID nvarchar(36),
            CONDITIONS_ID int,
            CARE_TYPE nvarchar(20),
            TABLE_ID int,
            foreign key(PATIENT_ID) references patients_new(Id) 
            foreign key(CONDITIONS_ID) references conditions_new(CONDITIONS_ID)
			)
             ''')             

In [82]:
cursor.execute('''
    INSERT INTO patients_new(Id, BIRTHDATE_YEAR, DEATHDATE,SSN,SUFFIX,MAIDEN,MARITAL,RACE,ETHNICITY,GENDER,BIRTHPLACE,
    CITY,STATE, COUNTY,ZIP, LAT,LON,HEALTHCARE_EXPENSES, HEALTHCARE_COVERAGE)
    SELECT  Id, strftime('%Y', BIRTHDATE) as BIRTHDATE_YEAR, DEATHDATE,SSN,SUFFIX,MAIDEN,MARITAL,RACE,ETHNICITY,GENDER,BIRTHPLACE,
    CITY,STATE, COUNTY,ZIP, LAT,LON,HEALTHCARE_EXPENSES, HEALTHCARE_COVERAGE 
    FROM patients
    ''') 

In [83]:
cursor.execute('''
    INSERT INTO conditions_new(START,STOP, PATIENT, ENCOUNTER,DESCRIPTION,CODE)
    SELECT START,STOP, PATIENT, ENCOUNTER,DESCRIPTION,CODE
    FROM conditions
    ''') 

In [84]:
cursor.execute('''
    INSERT INTO observations_new(DATE, PATIENT, ENCOUNTER,CODE, DESCRIPTION, UNITS)
    SELECT DATE, PATIENT, ENCOUNTER,CODE, DESCRIPTION, UNITS
    FROM observations
    ''')

In [85]:
cursor.execute('''
    INSERT INTO procedures_new(DATE, PATIENT, ENCOUNTER,CODE, DESCRIPTION, BASE_COST, REASONCODE,REASONDESCRIPTION )
    SELECT DATE, PATIENT, ENCOUNTER,CODE, DESCRIPTION, BASE_COST, REASONCODE,REASONDESCRIPTION
    FROM procedures
    ''')

In [86]:
cursor.execute('''
    INSERT INTO medications_new(START, STOP, PATIENT, PAYER, ENCOUNTER, CODE, DESCRIPTION, BASE_COST, PAYER_COVERAGE, DISPENSES, TOTALCOST, REASONCODE,REASONDESCRIPTION )
    SELECT START, STOP, PATIENT, PAYER, ENCOUNTER, CODE, DESCRIPTION, BASE_COST, PAYER_COVERAGE, DISPENSES, TOTALCOST, REASONCODE,REASONDESCRIPTION 
    FROM medications
    ''')


In [87]:
print(pd.read_sql_query('''
     SELECT *
     FROM patients_new
     ''', conn))

                                        Id  BIRTHDATE_YEAR DEATHDATE  \
0     d2061cc7-bee0-0e6c-3ac4-15c197c474e0            1956      None   
1     073d8e80-ff90-1c8d-57e4-29bfca52c87f            1964      None   
2     e1ff7e68-4097-9faf-514d-e4cfcfdf252e            1998      None   
3     a0f679cc-875f-dd72-ed13-9ca863ec6cf3            2004      None   
4     e4166a9c-f7c5-bef7-ea43-b96281a2d586            2006      None   
...                                    ...             ...       ...   
1014  2a41a25a-bb37-04db-4bd6-5a52474e309a            1991      None   
1015  a8bea34f-a8d3-f5e2-93d1-5e2e52307251            1952      None   
1016  8e1abff8-cf2e-ed15-1e93-753b6f2578ea            2002      None   
1017  f98799f2-c4da-761d-f3ac-5fac8e8fc8bc            1979      None   
1018  4d6c5c28-0195-8486-66bc-787fb1bc4099            1917      None   

              SSN SUFFIX         MAIDEN MARITAL   RACE    ETHNICITY GENDER  \
0     999-51-6528   None           None       M  white  n

In [88]:
## Faktentabelle

cursor.execute('''
    INSERT INTO patient_careplans(PATIENT_ID, CONDITIONS_ID, CARE_TYPE, TABLE_ID)
    SELECT 
    patients_new.Id, CONDITIONS_ID,  'Medications' as CARE_TYPE, medications_new.MEDICATIONS_ID
    FROM patients_new
    INNER JOIN 
    conditions_new on patients_new.Id = conditions_new.PATIENT
    INNER JOIN
    medications_new ON patients_new.Id = medications_new.PATIENT
    
    UNION

    SELECT 
    patients_new.Id, CONDITIONS_ID,  'Observations' as CARE_TYPE, observations_new.OBSERVATIONS_ID
    FROM patients_new
    INNER JOIN 
    conditions_new on patients_new.Id = conditions_new.PATIENT
    INNER JOIN
    observations_new ON patients_new.Id =  observations_new.PATIENT

    UNION

    SELECT 
    patients_new.Id, CONDITIONS_ID,  'Procedures' as CARE_TYPE, procedures_new.PROCEDURES_ID
    FROM patients_new
    INNER JOIN 
    conditions_new on patients_new.Id = conditions_new.PATIENT
    INNER JOIN
    procedures_new ON patients_new.Id =  procedures_new.PATIENT

    ''')

In [89]:
print(pd.read_sql_query('''
     SELECT *
     FROM patient_careplans
     ''', conn))

       CAREPLANT_ID                            PATIENT_ID  CONDITIONS_ID  \
0                 1  009121bf-a672-8942-443e-85e18a33f766             51   
1                 2  009121bf-a672-8942-443e-85e18a33f766             51   
2                 3  009121bf-a672-8942-443e-85e18a33f766             51   
3                 4  009121bf-a672-8942-443e-85e18a33f766             51   
4                 5  009121bf-a672-8942-443e-85e18a33f766             51   
...             ...                                   ...            ...   
74425         74426  fd6d5aae-f012-0e00-99bc-6668b73bf4f3            512   
74426         74427  fd6d5aae-f012-0e00-99bc-6668b73bf4f3            512   
74427         74428  fd6d5aae-f012-0e00-99bc-6668b73bf4f3            512   
74428         74429  fd6d5aae-f012-0e00-99bc-6668b73bf4f3            512   
74429         74430  fd6d5aae-f012-0e00-99bc-6668b73bf4f3            512   

          CARE_TYPE  TABLE_ID  
0      Observations      7190  
1      Observations    

In [90]:
print(pd.read_sql_query('''
     SELECT *
     FROM patient_careplans
     WHERE CAREPLANT_ID<10
     ''', conn))

   CAREPLANT_ID                            PATIENT_ID  CONDITIONS_ID  \
0             1  009121bf-a672-8942-443e-85e18a33f766             51   
1             2  009121bf-a672-8942-443e-85e18a33f766             51   
2             3  009121bf-a672-8942-443e-85e18a33f766             51   
3             4  009121bf-a672-8942-443e-85e18a33f766             51   
4             5  009121bf-a672-8942-443e-85e18a33f766             51   
5             6  009121bf-a672-8942-443e-85e18a33f766             51   
6             7  009121bf-a672-8942-443e-85e18a33f766             51   
7             8  009121bf-a672-8942-443e-85e18a33f766             51   
8             9  009121bf-a672-8942-443e-85e18a33f766             51   

      CARE_TYPE  TABLE_ID  
0  Observations      7190  
1  Observations      7191  
2  Observations      7192  
3  Observations      7193  
4  Observations      7194  
5  Observations      7195  
6  Observations      7196  
7  Observations      7197  
8  Observations      7198  

In [91]:
print(pd.read_sql_query('''
     SELECT CARE_TYPE
     FROM patient_careplans
     GROUP BY CARE_TYPE
     ''', conn))

      CARE_TYPE
0   Medications
1  Observations
2    Procedures


In [92]:
print(pd.read_sql_query('''
     SELECT PATIENT_ID, CARE_TYPE
     FROM patient_careplans
     WHERE CARE_TYPE = 'Medications'
     ''', conn))

                                PATIENT_ID    CARE_TYPE
0     02db7f22-8617-0cf7-fa10-d820d596a81a  Medications
1     02db7f22-8617-0cf7-fa10-d820d596a81a  Medications
2     02db7f22-8617-0cf7-fa10-d820d596a81a  Medications
3     02db7f22-8617-0cf7-fa10-d820d596a81a  Medications
4     02db7f22-8617-0cf7-fa10-d820d596a81a  Medications
...                                    ...          ...
7222  f5a6e3b3-781d-d800-d4c3-1782e39258e8  Medications
7223  f5a6e3b3-781d-d800-d4c3-1782e39258e8  Medications
7224  f5a6e3b3-781d-d800-d4c3-1782e39258e8  Medications
7225  f5a6e3b3-781d-d800-d4c3-1782e39258e8  Medications
7226  f5a6e3b3-781d-d800-d4c3-1782e39258e8  Medications

[7227 rows x 2 columns]


In [93]:
conn.close()